In [6]:
from watermark_model import Watermark
import torch
import numpy as np
import argparse
from scipy.stats import normaltest
from watermark_model import *
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_printoptions(sci_mode=False,profile='full')

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='decoder-encoder pretrain')
    parser.add_argument('--secret_length', default=48, type=int)
    args =parser.parse_known_args()[0]
    
model=Watermark(secret_length=args.secret_length).to(device)
model.load_state_dict(torch.load('./model48bit.pth'))
model.eval()

Metric=[]
Loss=[]
for i in range(100):
    secret=torch.Tensor(np.random.choice([0, 1], size=(args.secret_length))).to(device)
    secret = secret.unsqueeze(-1).unsqueeze(-1).unsqueeze(0)
    secret = secret.expand(-1,-1,64,64)
    y, mean, logvar = model(secret)

    mean=mean.reshape(-1,64,64)
    logvar=logvar.reshape(-1,64,64)
    eps = torch.randn_like(logvar)
    std = torch.exp(logvar / 2)
    matrix = eps * std + mean
    average_tensor1 = torch.mean(secret, dim=(-2, -1))
    average_tensor2 = torch.round(torch.mean(y[0], dim=(-2, -1)))
    average_tensor3 = torch.mean(y[0], dim=(-2, -1))
    # print(average_tensor1, average_tensor2, average_tensor3)

    biterror=torch.sum(abs(average_tensor1-average_tensor2))
    mse_loss = torch.nn.MSELoss()
    loss = mse_loss(average_tensor3, average_tensor1)
    Metric.append(biterror)
    Loss.append(loss)
    stat, p1 = normaltest(matrix.view(-1).cpu().detach().clone().numpy())
    print(p1)

biterror=torch.mean(torch.stack(Metric))
biterrper=biterror/args.secret_length
metric=torch.mean(torch.stack(Loss))
print(f'loss bits',f'{biterror.cpu().detach().numpy()}')
print(f'bit error percentage {biterrper*100:.2f}%')
print(f'loss',f'{metric.cpu().detach()}')

0.0008679440513399107
0.06282136975797975
0.3839916275684159
0.1221018706118531
0.5428503473503683
1.8632505162351127e-06
0.0019468245156236187
0.039870047342429084
0.2839882109245402
0.08614752468726301
0.006244370222610238
0.8673719762301627
0.006396033135268863
0.42112756007126806
0.4291568934893629
0.030823323497567022
0.3062076364117178
0.5501875570504399
0.06825874635251507
9.349193703460623e-06
0.1432962048097367
1.0463569687756277e-05
1.4510437099889853e-16
0.3387188707949777
0.3171837161942717
0.0010843952497042713
3.000944013425572e-08
0.012800910373206421
0.8719608595878682
0.23568207505814706
0.627101044623104
0.6510409803460866
0.0001264700808912938
0.011854906342118662
0.26118704397559306
1.9474733113192407e-185
0.06389457700177414
0.4209463173279797
0.06813871705818202
0.03219626735309046
0.8265682549862952
0.4175009354798219
0.1958457451222542
5.457798798671469e-08
0.5891521130168464
0.16829943096786484
2.8904422591958995e-11
0.695058377503931
0.6687014070893422
0.18975

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 